In [26]:
# import cv2    
# import numpy as np

# def picSuccessful(self_color):

#         area_coordinates = (300, 100, 100, 100)   #x,y,heigth,width
        
#         # Show the original image with the contour
#         image = cv2.imread("orange.png")
#         #image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)

#         # Extract the region of interest based on the provided coordinates
#         x, y, width, height = area_coordinates
#         roi = image[y:y+height, x:x+width]
        
#         # Convert the region of interest to the HSV color space
#         hsv_roi = cv2.cvtColor(roi, cv2.COLOR_RGB2HSV)
        
#         # Define color ranges for red, orange, yellow, and green
#         color_ranges = {
#             'red': ([0, 50, 50], [10, 255, 255]),
#             'orange': ([11, 50, 50], [25, 255, 255]),
#             'yellow': ([26, 50, 50], [34, 255, 255]),
#             'green': ([35, 50, 50], [70, 255, 255])
#         }




#         # farbe = (0, 255, 0)  # Green color (BGR format)
#         # thickness = 2  # Thickness of the rectangle border
#         # cv2.rectangle(image, (x, y), (x+width, y+height), farbe, thickness)

#         # # Display the image with the rectangle
#         # cv2.imshow('Image with Rectangle', image)

#         # # Create a window to display the image
#         # cv2.namedWindow('Image with Rectangle', cv2.WINDOW_NORMAL)
#         # cv2.imshow('Image with Rectangle', image)
#         # cv2.waitKey(0)
#         # cv2.destroyAllWindows()
        



#         # Check if the expected color is present in the region of interest
#         if self_color in color_ranges:
#             lower, upper = color_ranges[self_color]
#             lower = np.array(lower, dtype=np.uint8)
#             upper = np.array(upper, dtype=np.uint8)
#             mask = cv2.inRange(hsv_roi, lower, upper)
#             if cv2.countNonZero(mask) > 0:
#                 return True
            
#         return False



        # if self.color == "red":
        #     color_index = 2

        # if self.color == "green":
        #     color_index = 0

        # if self.color == "yellow":
        #     color_index = 3

        # if self.color == "orange":
        #     color_index = 1

In [27]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import os
import pyrealsense2 as rs
import tkinter as tk

from PIL import ImageTk

from PIL import Image
from scipy.interpolate import splprep, splev

from ultralytics import YOLO
import torch


def getYOLOMask():

    YOLO_IMG_PATH = "orange.png" # safe the image for YOLO here
    model = YOLO(model='/home/leon/Documents/Robologics/robologics_brause/src/robogistics_brause/robogistics_brause/object_detection/Yolov8_model/weights/best.pt')

    results = model.predict(source=YOLO_IMG_PATH, save=False, save_txt=False, stream=True)
    for result in results: # only on result in results
        
        # get array results
        all_masks = result.masks.data
        boxes = result.boxes.data
        color = boxes[:, 5] #
        idx_masks_color = torch.where(color == 1) # index of the chosen coulour
        # use these indices to extract the relevant masks
        for index in idx_masks_color:
            color_masks = all_masks[index]
            result_masks = [[],[]]
            for i in range(len(color_masks)):

                result_mask = color_masks[i].cpu().numpy()
                result_masks[0].append(np.count_nonzero(result_mask))
                result_masks[1].append(result_mask)

    _, final_mask_list = (list(t) for t in zip(*sorted(zip(result_masks[0], result_masks[1]), reverse=True)))

    mask_size = np.count_nonzero(final_mask_list[0])


    binary_array = np.where(final_mask_list[0] != 0, 1, 0)
    binary_array = binary_array.astype('uint8')
    resized_binary_array = cv2.resize(binary_array,(1280, 720))
    resized_binary_array.astype('uint8') # nicht sicher ob notwendig

    print("X", mask_size)

    return resized_binary_array, mask_size

getYOLOMask()


Ultralytics YOLOv8.0.20 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7982MiB)


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
